# Download the UCI Adults (US Census Bureau) dataset from the Database
Find two examples in the course content in Canvas  
Database Details:   
user='new_haven_ds_read'  
password='new_haven_ds_secret_99'  
host='nhds-fall-24.cwroivw0q1rc.us-east-1.rds.amazonaws.com'
database='nhds'  
  
  
 SQL: sql = 'select * from nhds.uci_adults'
 
 
 You can use either the py-postgresql or psycopg2 libraries.

In [11]:
import postgresql
db = postgresql.open("pq://new_haven_ds_read:new_haven_ds_secret_99@nhds-fall-24.cwroivw0q1rc.us-east-1.rds.amazonaws.com/nhds")

ClientCannotConnectError: could not establish connection to server
  CODE: 08001
  LOCATION: CLIENT
CONNECTION: [failed]
  failures[0]:
    SSL socket('54.159.108.37', 5432)
    Traceback (most recent call last):
      File "C:\Python312\Lib\site-packages\postgresql\protocol\client3.py", line 184, in connect
        self.socket = self.socket_factory.secure(self.socket)
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
      File "C:\Python312\Lib\site-packages\postgresql\python\socket.py", line 57, in secure
        return ssl.wrap_socket(socket, **self.socket_secure)
               ^^^^^^^^^^^^^^^
    AttributeError: module 'ssl' has no attribute 'wrap_socket'

    The above exception was the direct cause of the following exception:

    postgresql.exceptions.InsecurityError: SSL negotiation caused exception
      CODE: --SEC
      LOCATION: CLIENT
CONNECTOR: [Host] pq://new_haven_ds_read:***@nhds-fall-24.cwroivw0q1rc.us-east-1.rds.amazonaws.com:5432/nh

In [14]:
import psycopg2
connection_params = {
    'dbname': 'nhds',
    'user': 'new_haven_ds_read',
    'password': 'new_haven_ds_secret_99',
    'host': 'nhds-fall-24.cwroivw0q1rc.us-east-1.rds.amazonaws.com',
    'port': '5432'
}
conn = psycopg2.connect(**connection_params)
conn
cursor = conn.cursor()


### 1. How many rows are in the result set?

In [15]:
# ps = db.prepare("select * from nhds.uci_adults")
# rows = ps()
cursor.execute("select * from nhds.uci_adults;")
rows = cursor.fetchall()
print("Number of rows:", len(rows))

Number of rows: 32561


### 2. Create a DataFrame from the dataset

In [17]:
import pandas as pd

columns = [column.name for column in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,2
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,3
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,4
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560


### 3. How many columns are in the DataFrame?

In [18]:
df.shape[1]

16

### 4. What percentage of records show an individual making more than $50k/yr ?

In [25]:
df_above_50k = df[df['income'] == ' >50K']
print("percentage of individuals making more than $50k/year:", ((len(df_above_50k) / len(df)) * 100))

percentage of individuals making more than $50k/year: 24.080955744602438


### 5. Create a new column with an integer value representing income bucket (should be one of two values showing above/below 50k)

In [21]:
df['income_bucket'] = df['income'].apply(lambda x: 1 if x == '>50K' else 0)
df

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income,id,income_bucket
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,1,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,2,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,3,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,4,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K,32557,0
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K,32558,0
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K,32559,0
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K,32560,0


### 6. Pickle the dataset.  How much disk space does the pickled object use?

In [23]:
import os
filename = 'df.pkl'
df.to_pickle(filename)

file_size = os.path.getsize(filename) / 1024 #byte to KB
print(f"Pickled dataset size: {file_size:.2f} KB")

Pickled dataset size: 5703.18 KB


### 7. How many distinct values exist in the dataset for occupation ?

In [24]:
df['occupation'].nunique()

15

### 8. Explain why we practiced using the python pickle function in class.  Why would this be useful when working with large datasets?

to manage large datasets more effectively by reducing processing time and simplifying data handling

### 9. Using numPy compute the following:

<center>
<img src="../images/array_hw2.jpg" alt="drawing" style="width: 300px;"/>
</center>
    
    
    
    
    a) 2𝑉1 − 𝑉2
    b) 3𝑉1. 𝑉2
    c) 𝐴𝑣2
    d) 𝐴𝑇𝑣2

In [10]:
import numpy as np

v1 = np.array([2, 0, 1])
v2 = np.array([0.9, 5, -2])
A = np.array([
    [2, 3, 1],
    [1, -2, 4],
    [-3, 0, 5]
])

# a) 2𝑉1 − 𝑉2
print("2𝑉1 − 𝑉2: ", (2 * v1 - v2))

# b) 3𝑉1. 𝑉2
print("3𝑉1. 𝑉2: ", (3 * np.dot(v1, v2)))

# c) 𝐴𝑣2
print("𝐴𝑣2: ", (np.dot(A, v2)))

# d) 𝐴𝑇𝑣2
print("𝐴𝑇𝑣2: ", (np.dot(A.T, v2)))

2𝑉1 − 𝑉2:  [ 3.1 -5.   4. ]
3𝑉1. 𝑉2:  -0.5999999999999999
𝐴𝑣2:  [ 14.8 -17.1 -12.7]
𝐴𝑇𝑣2:  [12.8 -7.3 10.9]


### 10. Why do we avoid writing python loops when using pandas DataFrames?

for faster processing and memory usage we avoid loops in python